### using shared memory object previously created to run OnACID

In [2]:
import numpy as np
from multiprocessing.shared_memory import SharedMemory
import caiman as cm
from caiman.source_extraction import cnmf
from caiman.source_extraction.cnmf.online_cnmf import demix1p
from caiman.utils.nn_models import (fit_NL_model, create_LN_model, quantile_loss, rate_scheduler)

In [3]:
# Attach to the existing shared memory block
existing_shm = SharedMemory(name="shared_mem")

c = np.ndarray((2000,60,80), dtype=np.int16, buffer=existing_shm.buf)

In [4]:
c.nbytes

19200000

In [5]:
cnmf_shm = SharedMemory(name="cnmf_outputs", create=True, size=c.nbytes)

In [6]:
fname_init = "/home/clewis7/caiman_data/example_movies/demoMovie.tif"

In [7]:
cnmf_data = np.ndarray(c.shape, dtype=c.dtype, buffer=cnmf_shm.buf)
init_batch = 200  # number of frames to use for initialization
T = 2000           # total number of frames 
fr = 10 

In [8]:
params_dict = {'fnames': fname_init,
               'fr': fr,
               'method_init': 'corr_pnr',
               'K': None,
               'gSig': (3, 3),
               'gSiz': (13, 13),
               'merge_thr': .65,
               'p': 1,
               'tsub': 1,
               'ssub': 1,
               'only_init': True,
               'nb': 0,
               'min_corr': .65,
               'min_pnr': 6,
               'normalize_init': False,
               'ring_size_factor': 1.4,
               'center_psf': True,
               'ssub_B': 2,
               'init_iter': 1,
               's_min': -10,
               'init_batch': init_batch,
               'init_method': 'cnmf',
               'batch_update_suff_stat': True,
               'update_freq': 200,
               'expected_comps': 600,
               'update_num_comps': True,
               'rval_thr': .55,
               'thresh_fitness_raw': -130,
               'thresh_fitness_delta': -20,
               'min_SNR': 2.5,
               'min_num_trial': 5,
               'max_num_added': 5,
               'use_corr_img': False,
               'use_dense': False,
               'motion_correct': True,     # flag for performing motion correction
               'gSig_filt': (3, 3),        # size of high pass spatial filtering, used in 1p data
               'use_cnn': False}
opts = cnmf.params.CNMFParams(params_dict=params_dict)

In [9]:
cnm2 = cnmf.online_cnmf.OnACID(dview=None, params=opts)
cnm2.initialize_online(T=T)

/home/clewis7/venvs/mescore/lib/python3.9/site-packages/scipy/signal/_spectral_py.py:1999: UserWarning: nperseg = 256 is greater than input length  = 200, using nperseg = 200
  warnings.warn('nperseg = {0:d} is greater than input length '
/home/clewis7/venvs/mescore/lib/python3.9/site-packages/numpy/core/fromnumeric.py:43: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  result = getattr(asarray(obj), method)(*args, **kwds)
/home/clewis7/venvs/mescore/lib/python3.9/site-packages/scipy/signal/_spectral_py.py:1999: UserWarning: nperseg = 256 is greater than input length  = 200, using nperseg = 200
  warnings.warn('nperseg = {0:d} is greater than input length '
/home/clewis7/venvs/mescore/lib/python3.9/site-packages/scipy/signal/_spectral_py.py:1999: UserWarning: nperseg = 

In [10]:
cnmf_data = np.ndarray(c.shape, dtype=c.dtype, buffer=cnmf_shm.buf)

In [11]:
from multiprocessing import Process, Queue
from time import time, sleep

In [12]:
q = Queue()
cnm2.t_wait = []
cnm2.t_motion = []
cnm2.t_fit = []
realtime = []

def append_to_queue(q, init_batch, T, fr):
    t_start = time()
    for t in range(init_batch, T):
        # read frame and append to queue
        frame = c[t]
        q.put(frame)
        sleep(max(0, (t+1-init_batch)/fr - time()+t_start))

producer = Process(target=append_to_queue, args=(q,init_batch,T,fr))
producer.start()

# process first frame
t = init_batch
# read form queue (wait for next frame if empty)
t0=time()
frame = q.get()
cnm2.t_wait.append(time()-t0)
t_start = time() + (1-init_batch)/fr
# motion correct
t0 = time()
frame = cnm2.mc_next(t, frame)
cnm2.t_motion.append(time()-t0)
# fit
t0 = time()
cnm2.fit_next(t, frame.ravel(order='F'))
cnm2.t_fit.append(time()-t0)
rt = time() <= t_start + t/fr
realtime.append(rt)

# process remaining frames
for t in range(init_batch+1, T):
    # read form queue (wait for next frame if empty)
    t0=time()
    frame = q.get()
    cnm2.t_wait.append(time()-t0)
    # motion correct
    t0 = time()
    frame = cnm2.mc_next(t, frame)
    cnm2.t_motion.append(time()-t0)
    # fit
    t0 = time()
    cnm2.fit_next(t, frame.ravel(order='F'))
    cnm2.t_fit.append(time()-t0)
    rt = time() <= t_start + t/fr
    realtime.append(rt)
    # add code to display whatever you want in order to guide the closed-loop experiment below
    print('Realtime: ' + ("\x1b[32mTrue\x1b[0m" if rt else "\x1b[31mFalse\x1b[0m"), end="  \r", flush=True)

In [19]:
type(cnm2)

caiman.source_extraction.cnmf.online_cnmf.OnACID